In [84]:
import networkx as nx
import random as rand
import numpy as np
import matplotlib.pyplot as plt
from kernighan_lin import kernighan_lin_bisection
import networkx as nx
from stellargraph import StellarGraph
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE
from tensorflow.keras import layers, optimizers, losses, Model
import pandas as pd
import os


In [85]:
# We can get from timeclock
SEED_OF_RANDOM = 2524
rand.seed(SEED_OF_RANDOM)

In [86]:
def create_connected_graph(node_list):
    # Create an empty graph
    G = nx.Graph()

    # Add nodes from the specific node list
    G.add_nodes_from(node_list)

    # Add edges to connect the nodes and ensure connectivity
    while not nx.is_connected(G):
        # Randomly select two nodes from the graph
        node1 = rand.choice(list(G.nodes()))
        node2 = rand.choice(list(G.nodes()))

        # Add an edge between the selected nodes
        G.add_edge(node1, node2)

    return G

In [87]:
# simulate graph creation

def simulate_graph_creation(min_size, max_size, min_edge_between, max_edge_between):
        
    # Creating Graphs
    NUMBER_OF_NODES = rand.randint(min_size, max_size)
        
    nodeList_1 = np.arange(1, (NUMBER_OF_NODES+1)/2)
    nodeList_2 = np.arange((NUMBER_OF_NODES+1)/2, NUMBER_OF_NODES+1)
        
    connected_graph_1 = create_connected_graph(nodeList_1)
    connected_graph_2 = create_connected_graph(nodeList_2)
        
    RENAME_1 = 'G_1_' 
    RENAME_2 = 'G_2_'
    G_union = nx.union(connected_graph_1, connected_graph_2, rename = (RENAME_1, RENAME_2))
        
    # Setting Edges
    EDGES_BETWEEN_PARTITIONS = rand.randint(min_edge_between, max_edge_between)
        
    for i in range (EDGES_BETWEEN_PARTITIONS):
        node1 = RENAME_1 + str(rand.randint(1, NUMBER_OF_NODES))
        node2 = RENAME_2 + str(rand.randint(1, NUMBER_OF_NODES))
        G_union.add_edge(node1, node2)
        
    return G_union, EDGES_BETWEEN_PARTITIONS 

In [88]:
# labeling

def label_given_graph(graph, edges):
    
    partition = kernighan_lin_bisection(graph, max_iter = 200)
    
    G_partition1 = graph.subgraph(partition[0])
    G_partition2 = graph.subgraph(partition[1])
    
    # Check Vertex Constraint
    total_vertices = graph.number_of_nodes()
    partition_1_vertices = G_partition1.number_of_nodes()
    partition_2_vertices = G_partition2.number_of_nodes()
    
    min_vertex_bound = total_vertices/2 - total_vertices*0.01
    max_vertex_bound = total_vertices/2 + total_vertices*0.01
    
    if not ((min_vertex_bound <= partition_1_vertices <= max_vertex_bound) and (min_vertex_bound <= partition_2_vertices <= max_vertex_bound)):
        return False
    
    # Check Edge Constraint
    total_edges = graph.number_of_edges()
    partition_1_edges = G_partition1.number_of_edges()
    partition_2_edges = G_partition2.number_of_edges()

    min_cut_edge_amount = total_edges-partition_1_edges-partition_2_edges
    
    if edges >= min_cut_edge_amount:
        return True
    else:
        return False

In [89]:
# Generate node features
def generate_node_features(graph):
    # Calculate the degree of each node in the NodeView
    degrees = dict(graph.degree())
    
    # Create a DataFrame with node features
    node_features = pd.DataFrame.from_dict(degrees, orient="index", columns=["degree"])
    
    return node_features

In [90]:
# testing 

TEST_AMOUNT = 1

for i in range(TEST_AMOUNT):
    # Simulate graph creation
    G, edges = simulate_graph_creation(100, 500, 2, 5)

    # Label the graph
    label_val = label_given_graph(G, edges)

    # Assuming you have a function to generate node features, modify as needed
    node_features = generate_node_features(G)
    
    # Convert NetworkX graph to StellarGraph with node features
    G_stellar = StellarGraph.from_networkx(G, node_features=node_features)

    # GraphSAGE representation
    generator = GraphSAGENodeGenerator(G_stellar, batch_size=50, num_samples=[5, 5])
    model = GraphSAGE(layer_sizes=[50, 50], generator=generator, bias=True, dropout=0.5)
    x_inp, x_out = model.in_out_tensors()

    # Create a model for node embedding prediction
    prediction = layers.Dense(units=1, activation="sigmoid")(x_out)

    # Combine the GraphSAGE model with the prediction layer
    model = Model(inputs=x_inp, outputs=prediction)

    # Compile the model
    model.compile(optimizer=optimizers.Adam(lr=1e-3), loss=losses.binary_crossentropy, metrics=["acc"])

    # Obtain the graph embedding for all nodes
    node_ids = G_stellar.nodes()
    node_gen = generator.flow(node_ids)
    node_embeddings = model.predict(node_gen)

    # Map the representation with label_val (assuming label_val is boolean)
    mapping_result = dict(zip(node_ids, node_embeddings.flatten()))
    
    # Add the label information to mapping_result
    mapping_result['label'] = label_val

    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    file_path = os.path.join(desktop_path, "graph_mapping.xlsx")

    # Check if the file exists
    if os.path.exists(file_path):
        with pd.ExcelWriter(file_path, engine="openpyxl", mode="a") as writer:
            df = pd.DataFrame.from_dict(mapping_result, orient="index")
            startcol = len(writer.sheets["Sheet1"].columns) if "Sheet1" in writer.sheets else 0
            df.to_excel(writer, sheet_name="Sheet1", startcol=startcol, index=True)
    else:
        with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
            df = pd.DataFrame.from_dict(mapping_result, orient="index")
            df.to_excel(writer, sheet_name="Sheet1", index=True)